# Tinychat AppView

Collect messages from jetstream, dispatch them. What does an app view do? See
[this](https://github.com/bluesky-social/atproto/discussions/2961)

In [ ]:
//| export

// based on https://docs.deno.com/examples/chat_app_tutorial/

export default class ChatServer {
  private connectedClients = new Map<string, WebSocket>();

  public handleConnection(ws: WebSocket) {
    const id = `${Math.random() * 100000}`;

    ws.onclose = () => {
      this.clientDisconnected(id);
    };

    this.connectedClients.set(id, ws);
    console.log(">>>>>>> connectedClients", this.connectedClients.size);
  }

  private clientDisconnected(id: string) {
    this.connectedClients.delete(id);
    console.log(`Client ${id} disconnected`);
  }

  public broadcast(message: string) {
    for (const client of this.connectedClients.values()) {
      console.log(">>>>>>> sending message to", client);
      client.send(message);
    }
  }
}

In [ ]:
//| export

// patch HTMX WS extension to allow content inspection and modification

const htmxWS = `
/*
WebSockets Extension
============================
This extension adds support for WebSockets to htmx.  See /www/extensions/ws.md for usage instructions.
*/

(function() {
  /** @type {import("../htmx").HtmxInternalApi} */
  var api

  htmx.defineExtension('ws', {

    /**
     * init is called once, when this extension is first registered.
     * @param {import("../htmx").HtmxInternalApi} apiRef
     */
    init: function(apiRef) {
      // Store reference to internal API
      api = apiRef

      // Default function for creating new EventSource objects
      if (!htmx.createWebSocket) {
        htmx.createWebSocket = createWebSocket
      }

      // Default setting for reconnect delay
      if (!htmx.config.wsReconnectDelay) {
        htmx.config.wsReconnectDelay = 'full-jitter'
      }
    },

    /**
     * onEvent handles all events passed to this extension.
     *
     * @param {string} name
     * @param {Event} evt
     */
    onEvent: function(name, evt) {
      var parent = evt.target || evt.detail.elt
      switch (name) {
        // Try to close the socket when elements are removed
        case 'htmx:beforeCleanupElement':

          var internalData = api.getInternalData(parent)

          if (internalData.webSocket) {
            internalData.webSocket.close()
          }
          return

          // Try to create websockets when elements are processed
        case 'htmx:beforeProcessNode':

          forEach(queryAttributeOnThisOrChildren(parent, 'ws-connect'), function(child) {
            ensureWebSocket(child)
          })
          forEach(queryAttributeOnThisOrChildren(parent, 'ws-send'), function(child) {
            ensureWebSocketSend(child)
          })
      }
    }
  })

  function splitOnWhitespace(trigger) {
    return trigger.trim().split(/\s+/)
  }

  function getLegacyWebsocketURL(elt) {
    var legacySSEValue = api.getAttributeValue(elt, 'hx-ws')
    if (legacySSEValue) {
      var values = splitOnWhitespace(legacySSEValue)
      for (var i = 0; i < values.length; i++) {
        var value = values[i].split(/:(.+)/)
        if (value[0] === 'connect') {
          return value[1]
        }
      }
    }
  }

  /**
   * ensureWebSocket creates a new WebSocket on the designated element, using
   * the element's "ws-connect" attribute.
   * @param {HTMLElement} socketElt
   * @returns
   */
  function ensureWebSocket(socketElt) {
    // If the element containing the WebSocket connection no longer exists, then
    // do not connect/reconnect the WebSocket.
    if (!api.bodyContains(socketElt)) {
      return
    }

    // Get the source straight from the element's value
    var wssSource = api.getAttributeValue(socketElt, 'ws-connect')

    if (wssSource == null || wssSource === '') {
      var legacySource = getLegacyWebsocketURL(socketElt)
      if (legacySource == null) {
        return
      } else {
        wssSource = legacySource
      }
    }

    // Guarantee that the wssSource value is a fully qualified URL
    if (wssSource.indexOf('/') === 0) {
      var base_part = location.hostname + (location.port ? ':' + location.port : '')
      if (location.protocol === 'https:') {
        wssSource = 'wss://' + base_part + wssSource
      } else if (location.protocol === 'http:') {
        wssSource = 'ws://' + base_part + wssSource
      }
    }

    var socketWrapper = createWebsocketWrapper(socketElt, function() {
      return htmx.createWebSocket(wssSource)
    })

    socketWrapper.addEventListener('message', function(event) {
      if (maybeCloseWebSocketSource(socketElt)) {
        return
      }

      var response = event.data
      if (!api.triggerEvent(socketElt, 'htmx:wsBeforeMessage', {
        message: response,
        socketWrapper: socketWrapper.publicInterface
      })) {
        return
      }

      api.withExtensions(socketElt, function(extension) {
        response = extension.transformResponse(response, null, socketElt)
      })

      try {
        const d = JSON.parse(response);
        if (d && d.html) { response = d.html;}
      } catch (e) {
        // noop
      }

      var settleInfo = api.makeSettleInfo(socketElt)
      var fragment = api.makeFragment(response)

      if (fragment.children.length) {
        var children = Array.from(fragment.children)
        for (var i = 0; i < children.length; i++) {
          api.oobSwap(api.getAttributeValue(children[i], 'hx-swap-oob') || 'true', children[i], settleInfo)
        }
      }

      api.settleImmediately(settleInfo.tasks)
      api.triggerEvent(socketElt, 'htmx:wsAfterMessage', { message: response, socketWrapper: socketWrapper.publicInterface })
    })

    // Put the WebSocket into the HTML Element's custom data.
    api.getInternalData(socketElt).webSocket = socketWrapper
  }

  /**
   * @typedef {Object} WebSocketWrapper
   * @property {WebSocket} socket
   * @property {Array<{message: string, sendElt: Element}>} messageQueue
   * @property {number} retryCount
   * @property {(message: string, sendElt: Element) => void} sendImmediately sendImmediately sends message regardless of websocket connection state
   * @property {(message: string, sendElt: Element) => void} send
   * @property {(event: string, handler: Function) => void} addEventListener
   * @property {() => void} handleQueuedMessages
   * @property {() => void} init
   * @property {() => void} close
   */
  /**
   *
   * @param socketElt
   * @param socketFunc
   * @returns {WebSocketWrapper}
   */
  function createWebsocketWrapper(socketElt, socketFunc) {
    var wrapper = {
      socket: null,
      messageQueue: [],
      retryCount: 0,

      /** @type {Object<string, Function[]>} */
      events: {},

      addEventListener: function(event, handler) {
        if (this.socket) {
          this.socket.addEventListener(event, handler)
        }

        if (!this.events[event]) {
          this.events[event] = []
        }

        this.events[event].push(handler)
      },

      sendImmediately: function(message, sendElt) {
        if (!this.socket) {
          api.triggerErrorEvent()
        }
        if (!sendElt || api.triggerEvent(sendElt, 'htmx:wsBeforeSend', {
          message,
          socketWrapper: this.publicInterface
        })) {
          this.socket.send(message)
          sendElt && api.triggerEvent(sendElt, 'htmx:wsAfterSend', {
            message,
            socketWrapper: this.publicInterface
          })
        }
      },

      send: function(message, sendElt) {
        if (this.socket.readyState !== this.socket.OPEN) {
          this.messageQueue.push({ message, sendElt })
        } else {
          this.sendImmediately(message, sendElt)
        }
      },

      handleQueuedMessages: function() {
        while (this.messageQueue.length > 0) {
          var queuedItem = this.messageQueue[0]
          if (this.socket.readyState === this.socket.OPEN) {
            this.sendImmediately(queuedItem.message, queuedItem.sendElt)
            this.messageQueue.shift()
          } else {
            break
          }
        }
      },

      init: function() {
        if (this.socket && this.socket.readyState === this.socket.OPEN) {
          // Close discarded socket
          this.socket.close()
        }

        // Create a new WebSocket and event handlers
        /** @type {WebSocket} */
        var socket = socketFunc()

        // The event.type detail is added for interface conformance with the
        // other two lifecycle events (open and close) so a single handler method
        // can handle them polymorphically, if required.
        api.triggerEvent(socketElt, 'htmx:wsConnecting', { event: { type: 'connecting' } })

        this.socket = socket

        socket.onopen = function(e) {
          wrapper.retryCount = 0
          api.triggerEvent(socketElt, 'htmx:wsOpen', { event: e, socketWrapper: wrapper.publicInterface })
          wrapper.handleQueuedMessages()
        }

        socket.onclose = function(e) {
          // If socket should not be connected, stop further attempts to establish connection
          // If Abnormal Closure/Service Restart/Try Again Later, then set a timer to reconnect after a pause.
          if (!maybeCloseWebSocketSource(socketElt) && [1006, 1012, 1013].indexOf(e.code) >= 0) {
            var delay = getWebSocketReconnectDelay(wrapper.retryCount)
            setTimeout(function() {
              wrapper.retryCount += 1
              wrapper.init()
            }, delay)
          }

          // Notify client code that connection has been closed. Client code can inspect event field
          // to determine whether closure has been valid or abnormal
          api.triggerEvent(socketElt, 'htmx:wsClose', { event: e, socketWrapper: wrapper.publicInterface })
        }

        socket.onerror = function(e) {
          api.triggerErrorEvent(socketElt, 'htmx:wsError', { error: e, socketWrapper: wrapper })
          maybeCloseWebSocketSource(socketElt)
        }

        var events = this.events
        Object.keys(events).forEach(function(k) {
          events[k].forEach(function(e) {
            socket.addEventListener(k, e)
          })
        })
      },

      close: function() {
        this.socket.close()
      }
    }

    wrapper.init()

    wrapper.publicInterface = {
      send: wrapper.send.bind(wrapper),
      sendImmediately: wrapper.sendImmediately.bind(wrapper),
      reconnect: wrapper.init.bind(wrapper),
      queue: wrapper.messageQueue
    }

    return wrapper
  }

  /**
   * ensureWebSocketSend attaches trigger handles to elements with
   * "ws-send" attribute
   * @param {HTMLElement} elt
   */
  function ensureWebSocketSend(elt) {
    var legacyAttribute = api.getAttributeValue(elt, 'hx-ws')
    if (legacyAttribute && legacyAttribute !== 'send') {
      return
    }

    var webSocketParent = api.getClosestMatch(elt, hasWebSocket)
    processWebSocketSend(webSocketParent, elt)
  }

  /**
   * hasWebSocket function checks if a node has webSocket instance attached
   * @param {HTMLElement} node
   * @returns {boolean}
   */
  function hasWebSocket(node) {
    return api.getInternalData(node).webSocket != null
  }

  /**
   * processWebSocketSend adds event listeners to the <form> element so that
   * messages can be sent to the WebSocket server when the form is submitted.
   * @param {HTMLElement} socketElt
   * @param {HTMLElement} sendElt
   */
  function processWebSocketSend(socketElt, sendElt) {
    var nodeData = api.getInternalData(sendElt)
    var triggerSpecs = api.getTriggerSpecs(sendElt)
    triggerSpecs.forEach(function(ts) {
      api.addTriggerHandler(sendElt, ts, nodeData, function(elt, evt) {
        if (maybeCloseWebSocketSource(socketElt)) {
          return
        }

        /** @type {WebSocketWrapper} */
        var socketWrapper = api.getInternalData(socketElt).webSocket
        var headers = api.getHeaders(sendElt, api.getTarget(sendElt))
        var results = api.getInputValues(sendElt, 'post')
        var errors = results.errors
        var rawParameters = Object.assign({}, results.values)
        var expressionVars = api.getExpressionVars(sendElt)
        var allParameters = api.mergeObjects(rawParameters, expressionVars)
        var filteredParameters = api.filterValues(allParameters, sendElt)

        var sendConfig = {
          parameters: filteredParameters,
          unfilteredParameters: allParameters,
          headers,
          errors,

          triggeringEvent: evt,
          messageBody: undefined,
          socketWrapper: socketWrapper.publicInterface
        }

        if (!api.triggerEvent(elt, 'htmx:wsConfigSend', sendConfig)) {
          return
        }

        if (errors && errors.length > 0) {
          api.triggerEvent(elt, 'htmx:validation:halted', errors)
          return
        }

        var body = sendConfig.messageBody
        if (body === undefined) {
          var toSend = Object.assign({}, sendConfig.parameters)
          if (sendConfig.headers) { toSend.HEADERS = headers }
          body = JSON.stringify(toSend)
        }

        socketWrapper.send(body, elt)

        if (evt && api.shouldCancel(evt, elt)) {
          evt.preventDefault()
        }
      })
    })
  }

  /**
   * getWebSocketReconnectDelay is the default easing function for WebSocket reconnects.
   * @param {number} retryCount // The number of retries that have already taken place
   * @returns {number}
   */
  function getWebSocketReconnectDelay(retryCount) {
    /** @type {"full-jitter" | ((retryCount:number) => number)} */
    var delay = htmx.config.wsReconnectDelay
    if (typeof delay === 'function') {
      return delay(retryCount)
    }
    if (delay === 'full-jitter') {
      var exp = Math.min(retryCount, 6)
      var maxDelay = 1000 * Math.pow(2, exp)
      return maxDelay * Math.random()
    }

    logError('htmx.config.wsReconnectDelay must either be a function or the string "full-jitter"')
  }

  /**
   * maybeCloseWebSocketSource checks to the if the element that created the WebSocket
   * still exists in the DOM.  If NOT, then the WebSocket is closed and this function
   * returns TRUE.  If the element DOES EXIST, then no action is taken, and this function
   * returns FALSE.
   *
   * @param {*} elt
   * @returns
   */
  function maybeCloseWebSocketSource(elt) {
    if (!api.bodyContains(elt)) {
      var internalData = api.getInternalData(elt)
      if (internalData.webSocket) {
        internalData.webSocket.close()
        return true
      }
      return false
    }
    return false
  }

  /**
   * createWebSocket is the default method for creating new WebSocket objects.
   * it is hoisted into htmx.createWebSocket to be overridden by the user, if needed.
   *
   * @param {string} url
   * @returns WebSocket
   */
  function createWebSocket(url) {
    var sock = new WebSocket(url, [])
    sock.binaryType = htmx.config.wsBinaryType
    return sock
  }

  /**
   * queryAttributeOnThisOrChildren returns all nodes that contain the requested attributeName, INCLUDING THE PROVIDED ROOT ELEMENT.
   *
   * @param {HTMLElement} elt
   * @param {string} attributeName
   */
  function queryAttributeOnThisOrChildren(elt, attributeName) {
    var result = []

    // If the parent element also contains the requested attribute, then add it to the results too.
    if (api.hasAttribute(elt, attributeName) || api.hasAttribute(elt, 'hx-ws')) {
      result.push(elt)
    }

    // Search all child nodes that match the requested attribute
    elt.querySelectorAll('[' + attributeName + '], [data-' + attributeName + '], [data-hx-ws], [hx-ws]').forEach(function(node) {
      result.push(node)
    })

    return result
  }

  /**
   * @template T
   * @param {T[]} arr
   * @param {(T) => void} func
   */
  function forEach(arr, func) {
    if (arr) {
      for (var i = 0; i < arr.length; i++) {
        func(arr[i])
      }
    }
  }
})()
`;

In [ ]:
//| export

import { Hono } from "hono";
import { upgradeWebSocket } from "hono/deno";
// import { message } from "@tinychat/ui/message.tsx";
import { createMiddleware } from "hono/factory";
import { TinychatOAuthClient } from "tinychat/oauth.ts";
import { TinychatAgent } from "tinychat/agent.ts";

export type AppContext = {
  agent: () => Promise<TinychatAgent | undefined>;
};

export type HonoServer = Hono<{
  Variables: {
    ctx: AppContext;
  };
}>;

const app = new Hono();
const chatServer = new ChatServer();

app.use(
  "*",
  createMiddleware(async (c, next) => {
    const authorization = c.req.header("Authorization");
    const { client: oauthClient, user } = authorization
      ? await TinychatOAuthClient.fromAuthorizationHeader(authorization)
      : {};
    c.set("ctx", {
      oauthClient,
      session: undefined,
      agent: async () => await TinychatAgent.create(oauthClient, user),
    });
    await next();
  }),
);

app.get("/", (c) => c.redirect("https://github.com/callmephilip/tinychat"));

app.get("/__test", (c) =>
  c.html(`<!DOCTYPE html>
<html>
<head>
    <title>HTMX Chat</title>
    <script src="https://unpkg.com/htmx.org@2.0.4"></script>
    <script>${htmxWS}</script>
    <style>
        .chat-container { max-width: 600px; margin: 20px auto; }
        .messages { height: 400px; overflow-y: auto; border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; }
        .message { margin: 5px 0; padding: 5px; border-radius: 5px; background: #f0f0f0; }
        .input-form { display: flex; gap: 10px; }
        input { flex-grow: 1; padding: 5px; }
    </style>
</head>
<body>
    <div class="chat-container" hx-ext="ws" ws-connect="/ws">
        <div id="messages" class="messages">
        </div>
        <form class="input-form" ws-send>
            <input type="text" name="message" placeholder="Type a message..." autocomplete="off">
            <button type="submit">Send</button>
        </form>
    </div>
</body>
</html>`));

// app.get("/xrpc/chat.tinychat.getServers", async (c) => {
//   const ta = await c.var.ctx.agent();

//   if (!ta) {
//     return c.json({ error: "Unauthorized" }, 401);
//   }

//   await ta.chat.tinychat.server.create(
//     {
//       repo: ta.agent.assertDid,
//     },
//     {
//       name: "Test Servers",
//     }
//   );

//   return c.json({ message: "ok" });
// });

app.get(
  "/ws",
  upgradeWebSocket(() => {
    return {
      onOpen: (_, ws) => {
        if (!ws.raw) {
          return;
        }
        chatServer.handleConnection(ws.raw);
      },
    };
  }),
);

Hono {
  get: [Function (anonymous)],
  post: [Function (anonymous)],
  put: [Function (anonymous)],
  delete: [Function (anonymous)],
  options: [Function (anonymous)],
  patch: [Function (anonymous)],
  all: [Function (anonymous)],
  on: [Function (anonymous)],
  use: [Function (anonymous)],
  router: SmartRouter { name: "SmartRouter" },
  getPath: [Function: getPath],
  _basePath: "/",
  routes: [
    { path: "/*", method: "ALL", handler: [AsyncFunction (anonymous)] },
    { path: "/", method: "GET", handler: [Function (anonymous)] },
    { path: "/__test", method: "GET", handler: [Function (anonymous)] },
    {
      path: "/ws",
      method: "GET",
      handler: [AsyncFunction: UpgradeWebSocket]
    }
  ],
  errorHandler: [Function: errorHandler],
  onError: [Function: onError],
  notFound: [Function: notFound],
  fetch: [Function: fetch],
  request: [Function: request],
  fire: [Function: fire]
}

In [ ]:
//| export

import {
  NewChannelRecord,
  NewMembershipRecord,
  NewMessageRecord,
  NewServerRecord,
  startJetstream,
} from "tinychat/firehose.ts";
import { getProfile } from "tinychat/bsky.ts";
import { getDatabase } from "tinychat/db.ts";
import type { Database } from "tinychat/db.ts";

type AppViewShutdown = () => Promise<void>;
type AppViewContext = {
  database?: Database | undefined;
};

export const runAppView = (
  { database }: AppViewContext = {},
): AppViewShutdown => {
  const db = database || getDatabase();
  console.log("Starting appview with db", db);

  // Cleanup function
  const cleanup = () => {
    console.log("goodbye");
    Deno.removeSignalListener("SIGINT", cleanup);
    Deno.removeSignalListener("SIGTERM", cleanup);
    Deno.exit(0);
  };

  // Handle shutdown signals

  Deno.addSignalListener("SIGINT", cleanup);
  Deno.addSignalListener("SIGTERM", cleanup);

  console.log("Service started");

  const shutdownJetstream = startJetstream({
    onNewServer: async (m: NewServerRecord) => {
      const creator = m.did;
      const profile = await getProfile(creator);
      db.prepare(
        `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :displayName, :avatar, :description
        ) ON CONFLICT(did) DO UPDATE SET
          handle = COALESCE(:handle, handle),
          display_name = COALESCE(:displayName, display_name),
          avatar = COALESCE(:avatar, avatar),
          description = COALESCE(:description, description
        )`,
      ).run({
        did: m.did,
        handle: profile.handle,
        displayName: profile.displayName,
        avatar: profile.avatar,
        description: profile.description,
      });
      db.prepare(`
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )`).run({
        uri: m.uri,
        name: m.commit.record.name,
        creator: m.did,
      });
      db.prepare(
        `INSERT INTO server_memberships (user, server) VALUES (
          :creator, :server
        ) ON CONFLICT(user, server) DO NOTHING`,
      ).run({
        creator: m.did,
        server: m.uri,
      });
    },
    onNewChannel: (m: NewChannelRecord) => {
      db.prepare(
        `INSERT INTO channels (uri, name, server) VALUES (
          :uri, :name, :server
        ) ON CONFLICT(uri) DO NOTHING`,
      ).run({
        uri: m.uri,
        name: m.commit.record.name,
        server: m.commit.record.server,
      });
    },
    onNewMembership: (m: NewMembershipRecord) => {
      // add server memberships record
      try {
        db.prepare(
          `INSERT INTO server_memberships (user, server) VALUES (
          :creator, :server
        ) ON CONFLICT(user, server) DO NOTHING`,
        ).run({
          creator: m.did,
          server: m.commit.record.server,
        });
      } catch (e) {
        // normally this happens when creating a server and adding the creator to the server
        // membership gets processed before the server creation wraps up
        console.error("Error adding server membership", e);
      }
    },
    onNewMessage: (m: NewMessageRecord) => {
      db.prepare(
        `INSERT INTO messages (uri, channel, server, text, created_at) VALUES (
          :uri, :channel, :server, :text, :created_at
        )`,
      ).run({
        uri: m.uri,
        channel: m.commit.record.channel,
        server: m.commit.record.server,
        text: m.commit.record.text,
        created_at: m.commit.record.createdAt,
      });

      chatServer.broadcast(
        JSON.stringify({
          data: m.commit.record.text,
          html: `<div class="message">${m.commit.record.text}</div>`,
        }),
      );
    },
  });

  const server = Deno.serve(
    { port: parseInt(Deno.env.get("APPVIEW_PORT") || "8000") },
    app.fetch,
  );

  return async () => {
    try {
      shutdownJetstream();
      console.log("Shutting down server");
      await server.shutdown();
      console.log("Server shut down");
      Deno.removeSignalListener("SIGINT", cleanup);
      Deno.removeSignalListener("SIGTERM", cleanup);
    } catch (e) {
      console.error("Error shutting down server", e);
    }
  };
};

In [ ]:
const demo = () => {
  runAppView();

  return Deno.jupyter.html`
    <div style="padding: 20px; text-align: center;">
      <iframe
        width="800px"
        height="600px"
        src="http://localhost:8000"
      ></iframe>
    </div>
  `;
};

// Uncomment to run the demo
// demo();

## Test Appview

In [ ]:
import { TID } from "@atproto/common";
import { testClient } from "hono/testing";
import { assert, assertEquals } from "asserts";
import { sleep } from "tinychat/utils.ts";

Deno.test("/", async () => {
  // @ts-ignore cannot figure out type of test client
  const res = await testClient(app)["/"].$get();
  assertEquals(res.status, 302);
  // assertEquals(await res.json(), { status: "ok" });
});

// Deno.test("/xrpc/chat.tinychat.getServers", async () => {
//   // @ts-ignore cannot figure out type of test client
//   const res = await testClient(app)["/xrpc/chat.tinychat.getServers"].$get();
//   assertEquals(res.status, 200);
// });

Deno.test("test app view", async (t) => {
  const db = getDatabase();
  const shutdown = runAppView({ database: db });
  const serverName = `test-${TID.nextStr()}`;
  const agent = await TinychatAgent.create();
  const repo = await agent.agent.assertDid;
  const receivedMessages: { data: string; html: string }[] = [];

  // create websocket connection to chat server
  const clientWS = new WebSocket("ws://localhost:8001/ws");
  clientWS.onmessage = (event) => {
    receivedMessages.push(JSON.parse(event.data));
  };

  // let's create a new chat server and watch it propagate through the system
  // should see new elements synced with the db

  let server = "no server yet";
  let channel = "no channel yet";

  await t.step("create server", async () => {
    const chatServer = await agent.chat.tinychat.server.create(
      {
        repo,
      },
      {
        name: serverName,
      },
    );
    server = chatServer.uri;

    await agent.chat.tinychat.graph.membership.create(
      { repo },
      {
        server,
        createdAt: new Date().toISOString(),
      },
    );

    await sleep(2000);

    assert(
      db.prepare(`SELECT * FROM users`).all().length === 1,
      "user added to the db",
    );
    assert(
      db.prepare(`SELECT * FROM servers`).all().length === 1,
      "server added to the db",
    );
    assert(
      db.prepare(`SELECT * FROM server_memberships`).all().length === 1,
      "server membership added to the db",
    );
  });

  await t.step("create channel", async () => {
    const c = await agent.chat.tinychat.channel.create(
      { repo },
      {
        server,
        name: "general",
      },
    );
    channel = c.uri;

    await sleep(1000);

    assert(
      db.prepare(`SELECT * FROM channels`).all().length === 1,
      "channel added to the db",
    );
  });

  await t.step("send message", async () => {
    // add message
    await agent.chat.tinychat.message.create(
      { repo },
      {
        server,
        channel,
        text: "hello",
        createdAt: new Date().toISOString(),
      },
    );

    await sleep(1000);

    assert(
      db.prepare(`SELECT * FROM messages`).all().length === 1,
      "message added to the db",
    );
  });

  await t.step("create another server", async () => {
    await agent.chat.tinychat.server.create(
      { repo },
      { name: serverName + "2" },
    );

    await sleep(1000);

    assert(db.prepare(`SELECT * FROM servers`).all().length === 2);
  });

  await t.step("confirm messages get received over ws", () => {
    assert(receivedMessages.length === 1, "got one message");
    assert(
      typeof receivedMessages[0].data === "string",
      "message has a string data field",
    );
    assert(
      typeof receivedMessages[0].data === "string",
      "message has a string data field",
    );
  });

  // clean up and shutdown

  await shutdown();
  clientWS.close();
  await sleep(2000);
});